# Documentação Técnica: Extração e Processamento de BI-RADS - Fleury

## Objetivo Principal
**Este notebook realiza a extração, classificação e persistência dos dados de laudos de mamografia do Fleury, com foco na classificação BI-RADS.** O objetivo é identificar e classificar exames de mama segundo o BI-RADS, além de gerar uma lista de ativação para notificação de pacientes com exames em atraso.

## Tecnologias Utilizadas
- **PySpark**: Manipulação de dados, consultas SQL, transformações e persistência em Delta Lake.
- **Octoops**: Monitoramento, alertas e integração com sistemas de controle de execução.
- **Delta Lake**: Persistência dos dados processados.
- **Python (logging, traceback, etc.)**: Controle de fluxo, tratamento de erros e registro de eventos.

## Fluxo de Trabalho/Etapas Principais
1. Instalação de dependências (`octoops`).
2. Configuração do ambiente (reinicialização do kernel e importação de bibliotecas).
3. Definição de tabelas e filtros SQL para seleção incremental e ativação de pacientes.
4. Consulta SQL principal para extração e enriquecimento dos dados dos laudos.
5. Transformações nos DataFrames para cálculo de datas previstas de retorno, classificação e cálculo de diferenças de datas.
6. Persistência dos dados nas tabelas Delta, com tratamento de erros e envio de alertas.
7. Remoção de duplicados na lista de ativação.
8. Impressão da quantidade de registros salvos.

## Dados Envolvidos
- **Fonte**: Tabela `refined.saude_preventiva.fleury_laudos`.
- **Tabelas de destino**: `refined.saude_preventiva.fleury_laudos_mama_birads` e `refined.saude_preventiva.fleury_laudos_mama_birads_ativacao`.
- **Colunas importantes**: `laudo_tratado`, `BIRADS`, `MIN_BIRADS`, `MAX_BIRADS`, `dth_pedido`, `dth_previsao_retorno`, `dias_ate_retorno`, entre outras.

## Resultados/Saídas Esperadas
- DataFrame completo com laudos processados e classificados segundo BI-RADS.
- DataFrame filtrado para ativação de pacientes elegíveis para notificação.
- Persistência dos dados nas tabelas Delta.
- Alertas automáticos em caso de falhas ou ausência de dados.

## Pré-requisitos
- Ambiente Databricks ou Spark configurado.
- Pacotes: `octoops`, permissões de escrita nas tabelas Delta.
- Acesso às tabelas Delta e permissões de escrita.

## Considerações Importantes
- O notebook utiliza expressões regulares avançadas para correta identificação dos valores BI-RADS.
- O merge/upsert garante atualização incremental dos dados sem duplicidade.
- O monitoramento via Octoops/Sentinel facilita rastreabilidade e resposta rápida a falhas.
- O filtro de idade e sexo garante que apenas pacientes do público-alvo sejam analisados.

---
Cada célula de código abaixo é precedida por uma explicação técnica detalhada sobre sua função e impacto no fluxo do notebook.

### Instalação do pacote octoops
**Objetivo da Célula:** Instalar o pacote octoops diretamente do repositório privado do GitLab.

**Dependências:** Requer conectividade com o GitLab e credenciais de acesso (token) para acessar o repositório privado.

**Variáveis/Objetos Criados/Modificados:** Nenhum explicitamente, mas instala o pacote `octoops` no ambiente Python.

**Lógica Detalhada:** 
- Usa o comando pip com um parâmetro `--extra-index-url` que aponta para um repositório privado no GitLab.
- Utiliza credenciais (token) codificadas na URL para autenticação.
- O parâmetro `!` indica que o comando deve ser executado no sistema operacional, não no interpretador Python.

**Saída/Impacto:** Após execução, o pacote `octoops` estará disponível para importação no ambiente. Os logs da instalação serão exibidos.

In [ ]:
pip show octoops

In [ ]:
%pip install octoops==0.21.0

### Reinicialização do ambiente Python (Databricks)
**Objetivo da Célula:** Reiniciar o kernel Python do Databricks para garantir que as novas instalações de pacotes sejam carregadas corretamente.

**Dependências:** Requer o ambiente Databricks com a API `dbutils` disponível.


In [ ]:
dbutils.library.restartPython()

### Importação de bibliotecas
**Objetivo da Célula:** Importar todas as bibliotecas e funções necessárias para o processamento de dados, manipulação de datas e tratamento de erros.

**Dependências:** Requer que os pacotes PySpark e octoops estejam instalados no ambiente.

**Lógica Detalhada:**
- Importa funções específicas do módulo `pyspark.sql.functions` para manipulação de colunas e expressões SQL.
- Importa tipos de dados e classes para operações com DataFrames.
- Importa funções para manipulação de datas e cálculos de diferenças.
- Importa módulos Python padrão para logging e tratamento de exceções.
- Importa as classes do pacote octoops para monitoramento e alertas.


In [ ]:
from pyspark.sql.functions import col, year, month, dayofmonth, when, lit, expr, to_timestamp
from pyspark.sql.types import DateType
from pyspark.sql import DataFrame
from pyspark.sql.functions import datediff, to_date
from datetime import datetime
import logging
import sys
import traceback
from octoops import OctoOps
from octoops import Sentinel

### Configuração do logger
**Objetivo da Célula:** Configurar um logger para registrar eventos, avisos e erros durante a execução do notebook.

**Dependências:** Requer o módulo `logging` importado na célula anterior.

**Variáveis/Objetos Criados/Modificados:** Cria a variável `logger` como uma instância do objeto Logger.

**Lógica Detalhada:**
- Utiliza a função `logging.getLogger(__name__)` para criar um logger associado ao nome do módulo atual.
- O parâmetro `__name__` garante que o logger será específico para este notebook.
- Este logger será usado posteriormente para registrar mensagens informativas, avisos e erros.

**Saída/Impacto:** Cria e disponibiliza o objeto `logger` que será utilizado em outras funções do notebook para registrar informações de execução e erros.

In [ ]:
logger = logging.getLogger(__name__)

### Definição de tabelas e filtros
**Objetivo da Célula:** Definir os nomes das tabelas de destino e construir os filtros SQL para processamento incremental e ativação.

**Dependências:** Requer acesso ao catálogo Spark para verificação da existência de tabelas.

**Variáveis/Objetos Criados/Modificados:** 
- `table_birads`: nome da tabela principal de destino
- `table_birads_ativacao`: nome da tabela de destino para ativação
- `where_clause`: filtro SQL para seleção incremental
- `filtro_ativacao`: filtro SQL para regras de ativação

**Lógica Detalhada:**
1. Define os nomes das tabelas de destino no formato `schema.database.table`.
2. Inicializa `where_clause` como string vazia.
3. Verifica se a tabela principal já existe usando `spark.catalog.tableExists()`.
4. Se existir, cria um filtro SQL que seleciona apenas registros mais recentes que o último datestamp da tabela.
5. Define o filtro de ativação com regras de negócio específicas:
   - Sem ficha já existente (`eleg.ficha IS NULL`)
   - Apenas BI-RADS 1, 2 e 3
   - Apenas exames específicos de mamografia
   - Apenas pacientes do sexo feminino
   - Apenas pacientes na faixa etária de 40 a 75 anos

**Nomes de Tabelas/Colunas:**
- `refined.saude_preventiva.fleury_laudos_mama_birads`
- `refined.saude_preventiva.fleury_laudos_mama_birads_ativacao`
- Colunas filtradas: `ficha`, `BIRADS`, `sigla_exame`, `sexo_cliente`, `idade_cliente`

**Saída/Impacto:** Configura as variáveis com os nomes das tabelas e os filtros SQL que serão utilizados na consulta principal.

In [ ]:
# Tabela com todos os BIRADS extraídos dos laudos de mamografia
table_birads = "refined.saude_preventiva.fleury_laudos_mama_birads"

# Somente os laudos que estão dentro das regras de negócio para ativação (BIRADS 1, 2 e 3)
table_birads_ativacao = "refined.saude_preventiva.fleury_laudos_mama_birads_ativacao"
where_clause = ""
 
# datestamp => data em que recebemos os dados na plataforma
# Pegar da tabela de laudos
if spark.catalog.tableExists(table_birads):
    where_clause = f"""
    WHERE
        flr._datestamp >= (
            SELECT MAX(brd._datestamp)
            FROM {table_birads} brd
        )
    """
# Regra de negócio para ativação: BIRADS 1, 2 e 3, mulheres entre 40 e 75 anos, sem ficha ativa 
filtro_ativacao = """
    WHERE
        eleg.ficha IS NULL
        AND brd.BIRADS IN (1, 2, 3)
        AND flr.sigla_exame IN ('MAMOG', 'MAMOGDIG', 'MAMOPROT', 'MAMOG3D')
        AND UPPER(flr.sexo_cliente) = 'F'
        AND (
            idade_cliente >= 40 AND idade_cliente < 76
        )
"""

### Consulta SQL principal para extração e processamento de BI-RADS
**Objetivo da Célula:** Definir e executar a consulta SQL principal que extrai, limpa, classifica e enriquece os dados de laudos de mamografia.

**Dependências:**
- Variáveis `where_clause` e `filtro_ativacao` definidas anteriormente
- Acesso às tabelas: 
  - `refined.saude_preventiva.fleury_laudos`
  - `refined.saude_preventiva.fleury_retorno_elegivel_ficha`

**Variáveis/Objetos Criados/Modificados:**
- `query`: string da consulta SQL principal
- `df_spk`: DataFrame com todos os resultados da consulta
- `df_spk_ativacao`: DataFrame apenas com registros para ativação

**Lógica Detalhada:**
1. Define uma consulta SQL complexa com Common Table Expressions (CTEs):
   - `base`: Extrai BI-RADS dos laudos usando expressões regulares avançadas e transformações
   - `dados_birads`: Calcula valores mínimo, máximo e final do BI-RADS
   - `dados_laudos`: Seleciona e transforma dados principais dos laudos
2. A consulta principal:
   - Combina as CTEs com JOIN
   - Enriquece com dados de retorno elegível
   - Aplica os filtros dinâmicos `where_clause` e `filtro_ativacao`
3. Executa duas versões da consulta:
   - Primeira com filtro incremental mas sem filtro de ativação
   - Segunda sem filtro incremental mas com filtro de ativação

**Transformações principais:**
- Limpeza do texto com `REGEXP_REPLACE` removendo caracteres especiais
- Extração dos valores BI-RADS com `REGEXP_EXTRACT_ALL` em duas etapas
- Conversão de números romanos para decimais na função `TRANSFORM`
- Cálculo da idade usando `TIMESTAMPDIFF`
- Seleção dos valores mínimo/máximo de BI-RADS com funções de array

**Saída/Impacto:** 
- Cria dois DataFrames Spark: 
  - `df_spk` com todos os laudos relevantes processados
  - `df_spk_ativacao` apenas com os laudos que atendem critérios de ativação

In [ ]:
query = """
WITH base AS (
    SELECT
        flr.ficha,
        flr.id_item,
        flr.id_subitem,
        REGEXP_EXTRACT_ALL(
            REGEXP_EXTRACT(
                REGEXP_REPLACE(UPPER(flr.laudo_tratado), r'[-:®]|\xa0', ''),
                r'(?mi)(AVALIA[CÇ][AÃ]O|CONCLUS[AÃ]O|IMPRESS[AÃ]O|OPINI[AÃ]O)?(.*)', 2
            ),
            r"(?mi)(BIRADS|CATEGORIA|CAT\W)\s*(\d+\w*|VI|V|IV|III|II|I)\W*\w?(BIRADS|CATEGORIA)?(\W|$)", 2
        ) AS RAW_BIRADS,
        FILTER(
            TRANSFORM(RAW_BIRADS, x ->
                CASE
                    WHEN x = "I" THEN 1
                    WHEN x = "II" THEN 2
                    WHEN x = "III" THEN 3
                    WHEN x = "IV" THEN 4
                    WHEN x = "V" THEN 5
                    WHEN x = "VI" THEN 6
                    WHEN TRY_CAST(x AS INT) > 6 THEN NULL
                    ELSE REGEXP_REPLACE(x, r'[^0-9]', '')
                END
            ), x -> x IS NOT NULL
        ) AS CAT_BIRADS
    FROM refined.saude_preventiva.fleury_laudos flr
    WHERE
        flr.linha_cuidado = 'mama'
        AND flr.sigla_exame IN ('MAMOG', 'MAMOGDIG', 'MAMOPROT', 'MAMOG3D')
),
 
dados_birads AS (
    SELECT
        *,
        ARRAY_MIN(CAT_BIRADS) AS MIN_BIRADS,
        ARRAY_MAX(CAT_BIRADS) AS MAX_BIRADS,
        TRY_ELEMENT_AT(CAST(CAT_BIRADS AS ARRAY<INT>), -1) AS BIRADS
    FROM base
),
 
dados_laudos AS (
    SELECT
        flr.linha_cuidado,
        flr.id_unidade,
        flr.id_ficha,
        flr.id_item,
        flr.id_subitem,
        flr.ficha,
        flr.id_exame,
        flr.id_cliente,
        flr.pefi_cliente,
        flr.sigla_exame,
        flr.id_marca,
        flr.marca,
        (
          TIMESTAMPDIFF(DAY, flr.dth_nascimento_cliente, CURDATE()) / 365.25
        ) AS idade_cliente,
        flr.sexo_cliente,
        flr.dth_pedido,
        flr._datestamp
    FROM refined.saude_preventiva.fleury_laudos flr
    {where_clause}
)
 
SELECT
    flr.* except(idade_cliente),
    brd.MIN_BIRADS,
    brd.MAX_BIRADS,
    brd.BIRADS,
 
    eleg.dth_pedido        AS dth_pedido_retorno_elegivel,
    eleg.ficha             AS ficha_retorno_elegivel,
    eleg.siglas_ficha      AS siglas_ficha_retorno_elegivel,
    eleg.marca             AS marca_retorno_elegivel,
    eleg.unidade           AS unidade_retorno_elegivel,
    eleg.convenio          AS convenio_retorno_elegivel,
    eleg.valores_exame     AS valores_exame_retorno_elegivel,
    eleg.valores_ficha     AS valores_ficha_retorno_elegivel,
    eleg.qtd_exame         AS qtd_exame_retorno_elegivel,
    eleg.secao             AS secao_retorno_elegivel,
    eleg.dias_entre_ficha  AS dias_entre_ficha_elegivel
FROM dados_laudos flr
INNER JOIN dados_birads brd
    ON flr.ficha = brd.ficha
    AND flr.id_item = brd.id_item
    AND flr.id_subitem = brd.id_subitem
LEFT JOIN refined.saude_preventiva.fleury_retorno_elegivel_ficha eleg
    ON eleg.ficha_origem = flr.ficha
    AND eleg.id_cliente = flr.id_cliente
    AND eleg.linha_cuidado = flr.linha_cuidado
{filtro_ativacao}
"""

# Executa a consulta SQL com o filtro de _datestamp e sem o filtro de ativação
df_spk = spark.sql(query.format(
    where_clause = where_clause,
    filtro_ativacao = ""
    )
)
 
# Executa a consulta SQL sem o filtro de _datestamp e com o filtro de ativação 
df_spk_ativacao = spark.sql(query.format(
    where_clause = "",
    filtro_ativacao = filtro_ativacao
    )
)

### Função para calcular data prevista de retorno
**Objetivo da Célula:** Definir uma função que calcula a data prevista de retorno com base na classificação BI-RADS.

**Dependências:**
- Funções PySpark: `col`, `when`, `expr`
- Tipo de dados: `DataFrame`

**Função Criada:**
- `calcular_data_prevista(df_spk: DataFrame) -> DataFrame`

**Lógica Detalhada:**
1. A função recebe um DataFrame Spark como entrada.
2. Utiliza a função `withColumn` para adicionar uma nova coluna `dth_previsao_retorno`.
3. Aplica lógica condicional com `when` para diferentes valores de BI-RADS:
   - Para BI-RADS 1 ou 2: adiciona 360 dias à data de pedido
   - Para BI-RADS 3: adiciona 180 dias à data de pedido
   - Para outros valores: define como None (nulo)
4. Usa a função `expr("date_add(dth_pedido, N)")` para realizar o cálculo de datas.
5. Retorna o DataFrame com a nova coluna adicionada.

**Parâmetros:**
- `df_spk (DataFrame)`: O DataFrame Spark contendo os dados de entrada, que deve ter as colunas `BIRADS` e `dth_pedido`.

**Retorno:**
- `DataFrame`: O mesmo DataFrame de entrada, mas com a coluna adicional `dth_previsao_retorno`.

**Impacto:** Esta função implementa uma regra de negócio crucial para determinar quando o paciente deve retornar para um novo exame com base na classificação BI-RADS atual, afetando diretamente o fluxo de acompanhamento clínico.

In [ ]:
def calcular_data_prevista(df_spk: DataFrame):
    """
    Adiciona uma coluna 'dth_previsao_retorno' ao DataFrame com base na coluna 'BIRADS'.

    - Para BIRADS 1 ou 2, adiciona 360 dias à data da coluna 'dth_pedido'.
    - Para BIRADS 3, adiciona 180 dias à data da coluna 'dth_pedido'.
    - Para outros valores de BIRADS, define 'dth_previsao_retorno' como None.

    Parâmetros:
    df_spk (DataFrame): O DataFrame Spark contendo os dados de entrada.

    Retorna:
    DataFrame: O DataFrame atualizado com a nova coluna 'dth_previsao_retorno'.
    """
    df_spk = df_spk.withColumn(
        'dth_previsao_retorno',
        when(
            col('BIRADS').isin([1, 2]),
            expr("date_add(dth_pedido, 360)")
        ).when(
            col('BIRADS') == 3,
            expr("date_add(dth_pedido, 180)")  
        ).otherwise(None)
    )
    return df_spk

### Função para transformar campos do DataFrame
**Objetivo da Célula:** Definir uma função que aplica múltiplas transformações aos DataFrames para enriquecimento dos dados com base na classificação BI-RADS.

**Dependências:**
- Função `calcular_data_prevista` definida anteriormente
- Funções PySpark: `col`, `when`, `to_timestamp`, `datediff`, `to_date`
- Objeto `logger` para registro de avisos

**Função Criada:**
- `transform_fields(df_spk: DataFrame) -> DataFrame`

**Lógica Detalhada:**
1. Verifica se o DataFrame está vazio usando `isEmpty()`:
   - Se vazio, registra aviso e retorna o DataFrame sem alterações
2. Adiciona coluna `retorno_cliente` com a periodicidade em meses:
   - Para BI-RADS 1 ou 2: 12 meses
   - Para BI-RADS 3: 6 meses
   - Para outros valores: 0
3. Chama a função `calcular_data_prevista` para adicionar a coluna de data prevista
4. Converte duas colunas de data para o tipo timestamp:
   - `dth_pedido_retorno_elegivel`
   - `dth_previsao_retorno`
5. Calcula a diferença em dias entre a data prevista e a data do pedido na coluna `dias_ate_retorno`

**Parâmetros:**
- `df_spk (DataFrame)`: O DataFrame a ser transformado

**Retorno:**
- `DataFrame`: O DataFrame com todas as transformações aplicadas

**Colunas Adicionadas/Modificadas:**
- `retorno_cliente`: número de meses para retorno
- `dth_previsao_retorno`: data calculada para retorno
- `dias_ate_retorno`: número de dias até a data de retorno
- Conversões de tipo nas colunas de data

**Impacto:** Esta função enriquece o DataFrame com informações cruciais para o agendamento e acompanhamento de pacientes, calculando prazos e convertendo dados para formatos apropriados.

In [ ]:
def transform_fields(df_spk: DataFrame) -> DataFrame:
    """
    Transforma os campos do DataFrame fornecido.

    - Verifica se o DataFrame está vazio. Se estiver, registra um aviso e retorna o DataFrame sem alterações.
    - Adiciona uma coluna 'retorno_cliente' com valores baseados na coluna 'BIRADS':
      - 12 meses para BIRADS 1 ou 2
      - 6 meses para BIRADS 3
      - 0 para outros valores
    - Calcula a data prevista de retorno usando a função 'calcular_data_prevista'.
    - Converte a coluna 'dth_pedido_retorno_elegivel' para o tipo timestamp.
    - Converte a coluna 'dth_previsao_retorno' para o tipo timestamp.
    - Calcula a diferença em dias entre 'dth_previsao_retorno' e 'dth_pedido', armazenando o resultado na coluna 'dias_ate_retorno'.

    Parâmetros:
    df_spk (DataFrame): O DataFrame a ser transformado.

    Retorna:
    DataFrame: O DataFrame transformado com as novas colunas.
    """
    if df_spk.isEmpty():
        logger.warning("No Data Found!")
        return df_spk
    
    df_spk = df_spk.withColumn(
        'retorno_cliente',
        when(col('BIRADS').isin([1, 2]), 12).when(col('BIRADS') == 3, 6).otherwise(0)
    )

    df_spk = calcular_data_prevista(df_spk)
    df_spk = df_spk.withColumn('dth_pedido_retorno_elegivel', to_timestamp(col('dth_pedido_retorno_elegivel')))
    df_spk = df_spk.withColumn('dth_previsao_retorno', to_timestamp(col('dth_previsao_retorno')))
    df_spk = df_spk.withColumn('dias_ate_retorno', datediff(to_date(col('dth_previsao_retorno')), to_date(col('dth_pedido'))))
    return df_spk

### Configuração da função de tratamento de erros e alerta
**Objetivo da Célula:** Definir uma variável de ambiente e uma função para tratamento de erros com envio de alertas ao sistema Sentinel.

**Dependências:**
- Módulos: `traceback`
- Classes: `Sentinel` do pacote octoops

**Variáveis/Funções Criadas:**
- `WEBHOOK_DS_AI_BUSINESS_STG`: constante que define o ambiente ('prd')
- `error_message(e)`: função para tratamento de exceções

**Lógica Detalhada:**
1. Define a constante `WEBHOOK_DS_AI_BUSINESS_STG` com valor 'prd' (produção)
2. A função `error_message` recebe uma exceção `e` e:
   - Formata o traceback completo da exceção usando `traceback.format_exc()`
   - Constrói uma mensagem de erro formatada
   - Cria uma instância da classe `Sentinel` configurada para o projeto 'Monitor_Linhas_Cuidado_Mama'
   - Envia um alerta ao Sentinel usando `alerta_sentinela()`
   - Imprime o traceback no console usando `traceback.print_exc()`
   - Relança a exceção original para propagar o erro

**Parâmetros da Função:**
- `e (Exception)`: A exceção capturada que será processada e relatada

**Impacto:** Esta configuração implementa um mecanismo robusto de tratamento de erros, garantindo que qualquer falha durante a persistência dos dados será:
1. Registrada em log com detalhes completos
2. Notificada ao sistema de monitoramento (Sentinel)
3. Propagada adequadamente para tratamento em níveis superiores

Isso é crucial para manter a qualidade e confiabilidade do pipeline de dados, facilitando o diagnóstico e resposta rápida a problemas.

In [ ]:
WEBHOOK_DS_AI_BUSINESS_STG = 'prd'

def error_message(e):
    """
    Envia alerta para o Sentinel e exibe o traceback em caso de erro ao salvar dados.

    Parâmetros:
        e (Exception): Exceção capturada.

    Comportamento:
        - Formata o traceback do erro.
        - Envia alerta para o Sentinel com detalhes do erro.
        - Exibe o traceback no console.
        - Relança a exceção.
    """
    error_message = traceback.format_exc()
    summary_message = f"""Erro ao salvar dados.\n{error_message}"""
    sentinela_ds_ai_business = Sentinel(
        project_name='Monitor_Linhas_Cuidado_Mama',
        env_type=WEBHOOK_DS_AI_BUSINESS_STG,
        task_title='Fleury Mamografia'
    )
    sentinela_ds_ai_business.alerta_sentinela(
        categoria='Alerta', 
        mensagem=summary_message,
        job_id_descritivo='1_fleury_mama_birads'
    )
    traceback.print_exc()
    raise e

### Funções de persistência de dados em tabelas Delta
**Objetivo da Célula:** Definir três funções para persistência de dados em tabelas Delta, com suporte a inserção, merge e decisão inteligente entre essas operações.

**Dependências:**
- Função `error_message` definida anteriormente
- Objeto `logger` para registros
- API Spark SQL para execução de operações SQL

**Funções Criadas:**
1. `insert_data(df_spk, table_name, serializable=False)`: Insere dados em uma tabela Delta, sobrescrevendo dados existentes
2. `merge_data(df_spk, table_name)`: Realiza um merge (upsert) em uma tabela Delta existente
3. `save_data(df_spk, table_name, serializable=False)`: Função principal que decide entre insert ou merge

**Lógica Detalhada:**

**1. `insert_data`:**
   - Registra o início da operação com `logger.info`
   - Define o nível de isolamento da transação conforme parâmetro `serializable`
   - Utiliza a API de escrita do Spark para salvar o DataFrame:
     - Formato 'delta'
     - Permite sobrescrita do schema
     - Define o modo de isolamento da transação
     - Usa modo 'overwrite' para substituir dados
   - Captura exceções e chama `error_message` em caso de erro

**2. `merge_data`:**
   - Registra o início da operação com `logger.info`
   - Cria uma view temporária com os dados do DataFrame
   - Executa uma operação MERGE SQL:
     - Combina dados com base nas chaves (ficha, id_item, id_subitem)
     - Atualiza registros existentes
     - Insere novos registros
   - Captura exceções e chama `error_message` em caso de erro

**3. `save_data`:**
   - Verifica se o DataFrame está vazio - se estiver, retorna sem fazer nada
   - Verifica se a tabela já existe no catálogo Spark
   - Se existir, chama `merge_data` para fazer upsert
   - Se não existir, chama `insert_data` para criar uma nova tabela

**Impacto:** Este conjunto de funções implementa uma estratégia robusta de persistência de dados que:
1. Evita processamento desnecessário para DataFrames vazios
2. Decide automaticamente entre criação de tabela ou atualização incremental
3. Gerencia corretamente o isolamento das transações
4. Fornece tratamento de exceções e alertas em caso de falha
5. Mantém registros de log para rastreabilidade das operações

In [ ]:
def insert_data(df_spk: DataFrame, table_name: str, serializable: bool = False):
    """
    Insere os dados do DataFrame em uma tabela Delta no Databricks.
    
    Parâmetros:
        df_spk (DataFrame): DataFrame Spark a ser salvo.
        table_name (str): Nome da tabela destino.
        serializable (bool): Define o nível de isolamento da transação Delta. 
            Se True, usa 'Serializable', caso contrário 'WriteSerializable'.
    
    Comportamento:
        - Escreve o DataFrame na tabela Delta especificada, sobrescrevendo dados existentes.
        - Atualiza o schema da tabela conforme necessário.
        - Define o nível de isolamento da transação Delta.
    """
    try:
        logger.info(f"Inserting Data: {table_name}")
        isolation_level = 'Serializable' if serializable else 'WriteSerializable'
        (
            df_spk.write
                .format('delta')
                .option('overwriteSchema', 'true')
                .option('delta.isolationLevel', isolation_level)
                .mode('overwrite')
                .saveAsTable(table_name)
        )
    except Exception as e:
        error_message(e)
 
def merge_data(df_spk: DataFrame, table_name: str):
    """
    Realiza um merge (upsert) dos dados do DataFrame em uma tabela Delta existente.
    
    Parâmetros:
        df_spk (DataFrame): DataFrame Spark a ser mesclado.
        table_name (str): Nome da tabela destino.
    
    Comportamento:
        - Cria uma view temporária com os dados do DataFrame.
        - Executa um comando MERGE SQL para atualizar registros existentes (com base em ficha, id_item e id_subitem)
          ou inserir novos registros na tabela Delta.
    """
    try:
        logger.info(f"Merging Data: {table_name}")
        df_spk.createOrReplaceTempView(f"increment_birads")
        merge_query = f"""
            MERGE INTO {table_name} AS target
            USING increment_birads AS source
                ON target.ficha = source.ficha
                AND target.id_item = source.id_item
                AND target.id_subitem = source.id_subitem
            WHEN MATCHED THEN
                UPDATE SET *
            WHEN NOT MATCHED THEN
                INSERT *
        """
        spark.sql(merge_query)
    except Exception as e:
        error_message(e)
 
def save_data(df_spk: DataFrame, table_name: str, serializable: bool = False):
    """
    Salva os dados do DataFrame em uma tabela Delta, realizando merge se a tabela já existir.
    
    Parâmetros:
        df_spk (DataFrame): DataFrame Spark a ser salvo.
        table_name (str): Nome da tabela destino.
        serializable (bool): Define o nível de isolamento da transação Delta na inserção inicial.
    
    Comportamento:
        - Se o DataFrame estiver vazio, não faz nada.
        - Se a tabela já existe, realiza merge (upsert) dos dados.
        - Se a tabela não existe, insere os dados criando a tabela.
    """
    if df_spk.isEmpty():
        return None
 
    if spark.catalog.tableExists(table_name):
        merge_data(df_spk, table_name)
    else:
        insert_data(df_spk, table_name, serializable)

### Aplicação das transformações aos DataFrames
**Objetivo da Célula:** Aplicar as transformações definidas anteriormente aos dois DataFrames extraídos da consulta SQL.

**Dependências:**
- Função `transform_fields` definida anteriormente
- DataFrames `df_spk` e `df_spk_ativacao` criados pela consulta SQL

**Variáveis/Objetos Modificados:**
- `df_spk`: DataFrame completo com todos os laudos
- `df_spk_ativacao`: DataFrame com apenas laudos elegíveis para ativação

**Lógica Detalhada:**
- Chama a função `transform_fields()` para cada DataFrame
- A função aplica as seguintes transformações (conforme definida anteriormente):
  1. Adiciona coluna `retorno_cliente` (12 meses para BI-RADS 1/2, 6 meses para BI-RADS 3)
  2. Calcula data prevista de retorno (360 dias para BI-RADS 1/2, 180 dias para BI-RADS 3)
  3. Converte colunas de datas para timestamp
  4. Calcula diferença em dias entre datas

**Saída/Impacto:** 
- Ambos os DataFrames são enriquecidos com as colunas calculadas necessárias para análise e persistência
- Os DataFrames mantêm seus filtros específicos:
  - `df_spk`: todos os registros, possivelmente filtrados por datestamp
  - `df_spk_ativacao`: apenas registros elegíveis para ativação

In [ ]:
# Aplicar transformações nos dfs
df_spk = transform_fields(df_spk)
df_spk_ativacao = transform_fields(df_spk_ativacao)

### Remoção de duplicados na lista de ativação
**Objetivo da Célula:** Remover registros duplicados do DataFrame de ativação, garantindo que exista apenas um registro por ficha.

**Motivação:** 
- Conforme comentado no código, o objetivo é enviar apenas 1 push (notificação) por ficha
- Se não removesse os duplicados, poderia haver múltiplas notificações para o mesmo atendimento quando há vários exames relacionados

In [ ]:
# Excluir duplicados para considerar apenas a ficha na ativação e não os exames (itens). Assim vamos enviar apenas 
# 1 push por ficha
df_spk_ativacao = df_spk_ativacao.dropDuplicates(['ficha'])

### Impressão da quantidade de registros
**Objetivo da Célula:** Exibir a quantidade de registros em cada DataFrame para monitoramento do volume de dados processados.

In [ ]:
print('quantidade de laudos salvos na tabela',df_spk.count())
print('quantidade de laudos salvos na tabela de ativação', df_spk_ativacao.count())

### Persistência dos dados nas tabelas Delta
**Objetivo da Célula:** Salvar os dados processados nas tabelas Delta correspondentes, utilizando as funções de persistência apropriadas.

**Dependências:**
- Funções `save_data` e `insert_data` definidas anteriormente
- DataFrames `df_spk` e `df_spk_ativacao` completamente processados
- Variáveis `table_birads` e `table_birads_ativacao` com nomes das tabelas

**Operações Executadas:**
1. Para o DataFrame principal `df_spk`:
   - Utiliza `save_data()`, que decide entre merge ou insert baseado na existência da tabela
   - Salva na tabela definida em `table_birads`

2. Para o DataFrame de ativação `df_spk_ativacao`:
   - Utiliza `insert_data()` diretamente, que sempre sobrescreve os dados existentes
   - Salva na tabela definida em `table_birads_ativacao`

**Lógica da Diferença de Abordagem:**
- Para dados completos (`df_spk`): usa abordagem incremental (merge) se a tabela já existir
- Para dados de ativação (`df_spk_ativacao`): sempre sobrescreve, pois representa o estado atual das ativações necessárias

**Comentários no Código:**
- O primeiro comentário explica a lógica do `save_data` (verifica se DataFrame está vazio, decide entre merge/insert)
- O segundo comentário explica a lógica do `insert_data` (sobrescreve dados, atualiza schema, define isolamento)

**Saída/Impacto:**
- Os dados processados são persistidos nas tabelas Delta correspondentes
- As tabelas ficam disponíveis para consulta por outros notebooks e aplicações
- Em caso de erro, o mecanismo de alerta do Sentinel será acionado

In [ ]:
# - Se o DataFrame estiver vazio, não faz nada.
# - Se a tabela já existe, realiza merge (upsert) dos dados.
# - Se a tabela não existe, insere os dados criando a tabela.
save_data(df_spk, table_birads)

# - Escreve o DataFrame na tabela Delta especificada, sobrescrevendo dados existentes.
# - Atualiza o schema da tabela conforme necessário.
# - Define o nível de isolamento da transação Delta.
insert_data(df_spk_ativacao, table_birads_ativacao)